In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from numpy.ma.core import count
from prompt_toolkit.key_binding.bindings.named_commands import yank_pop

warnings.filterwarnings('ignore')

In [171]:
bd = pd.read_csv('final.csv')
bd['finish_dttm'] = pd.to_datetime(bd['finish_dttm'])
bd['age'] = bd['age'].str.replace(',', '.').astype(float)
bd['flag'] = (bd['wo_hit_status_result_desc'] == 'Дозвон, Успешно')
bd['flag'] = bd['flag'].astype(int)

days_of_week = bd['finish_dttm'].dt.day_name()
days_of_week_num = bd['finish_dttm'].dt.weekday
bd['day_of_week'] = days_of_week
bd['num_of_week'] = days_of_week_num

def vibrosi(bd, i, x):
    return pd.concat([bd[bd[i] < x], bd[bd[i].isna()]])
def vibrosiVniz(bd, i, x):
    return pd.concat([bd[bd[i] > x], bd[bd[i].isna()]])
def percentsByRegion(bd, i):
    df = bd.copy()
    pt = df[df['region_size'] == i][df['age'] >= 18][df['age'] < 60][['age', 'flag']].pivot_table(index='age', columns='product_cd', values='flag', aggfunc=['sum', 'count'])
    # pt['percent'] = pt[(  'sum', 'flag')] / pt[('count', 'flag')] * 100
    # pt[pt['percent'] != 0]

bd['child_flg'] = bd['child_amt'] != 0
bd['child_flg'] = bd['child_flg'].astype(int)

bd = vibrosi(bd, 'confirmed_income', 40000)
bd = vibrosi(bd, 'predicted_income', 45000)
bd = vibrosiVniz(bd, 'predicted_income', 0)
bd = vibrosiVniz(bd, 'age', 13)
# хочу делить по: age, auto_flg, partnership_flg, child_flg, realty_type, functional_cd, emp_work_months_cnt, emp_gender_cd, emp_age

df = bd.drop(bd[bd['wo_hit_status_reason_desc'].isin(['Отказ прослушать предложение', 'Не звонить', 'Левый контакт', 'Отказ от диалога', 'отказ от диалога', 'Отказ от разговора', 'Не звонить, резкий негатив', 'Негатив/Не звонить', 'Отказ без объяснения причин', 'просит не звонить', 'Не звонить / резкий негатив', 'Вежливо попросил не звонить', 'Номер не принадлежит страхователю', 'отказ без объяснения причин', 'Отказался озвучить причину', 'Чёрный список', 'негатив к Тинькофф', 'Грубо попросил не звонить', 'Не звонить (негатив)', 'Отказался от диалога', 'Просит больше не звонить', 'Недозвон', 'некорректная заявка', 'Техническая ошибка', 'Не нравится Т-Банк', 'Негатив к Т-Банку', 'Негативная информация о клиенте', 'Отказ без объяснения причины', 'Неизвестен / Не контактирует', 'Человек не участвовал в Дано', 'отменили'])].index)

df['income'] = df['confirmed_income'].fillna(df['predicted_income'])
df['segment_number'] = df['segment_cd'].str.extract(r'\((\d+)\)').astype(float)
df['num_segment'] = (df['segment_cd'].str[2:4])
df['num_segment'] = df['num_segment'].astype(float)

# Функция для определения сезона по номеру месяца
def get_season(month):
    if month in [12, 1, 2]:
        return 'Зима'
    elif month in [3, 4, 5]:
        return 'Весна'
    elif month in [6, 7, 8]:
        return 'Лето'
    elif month in [9, 10, 11]:
        return 'Осень'

# Преобразуем finish_dttm в datetime, если нужно
df['finish_dttm'] = pd.to_datetime(df['finish_dttm'])

# Извлекаем номер месяца
df['month'] = df['finish_dttm'].dt.month
df['hours'] = df['finish_dttm'].dt.hour

# Применяем функцию для создания нового столбца season
df['season'] = df['month'].apply(get_season)


In [172]:
emp = df[['employee_id', 'emp_age', 'emp_age_new', 'emp_gender_cd', 'emp_children_cnt', 'emp_work_months_cnt', 'state_talk_time_sec', 'flag']].groupby(by=['employee_id', 'emp_gender_cd']).mean()
emp.reset_index(inplace=True)
emp['time_uspeh'] = 0

def ok(id):
    return df[df['flag'] == 1][df['employee_id'] == id]['state_talk_time_sec'].mean()

emp['time_uspeh'] = emp['employee_id'].apply(ok)

def min(id):
    if len(df[df['flag'] == 1][df['employee_id'] == id]['employee_id'].tolist()) == 0:
        return 0
    return np.percentile(df[df['flag'] == 1][df['employee_id'] == id]['age'], 25)

def max(id):
    if len(df[df['flag'] == 1][df['employee_id'] == id]['employee_id'].tolist()) == 0:
        return 0
    return np.percentile(df[df['flag'] == 1][df['employee_id'] == id]['age'], 75)

emp['min_age'] = emp['employee_id'].apply(min)
emp['max_age'] = emp['employee_id'].apply(max)

# 1. Считаем успешность по employee_id и gender_cd
success_rates = (
    df.groupby(['employee_id', 'gender_cd'])['flag']
    .mean()
    .reset_index()
    .rename(columns={'flag': 'success_rate'})
)

# 2. Для каждого employee_id выбираем gender_cd с максимальной успешностью
best_gender_per_employee = (
    success_rates.loc[success_rates.groupby('employee_id')['success_rate'].idxmax()]
    [['employee_id', 'gender_cd']]
    .rename(columns={'gender_cd': 'lovely_gender_cd'})
)

emp = emp.merge(best_gender_per_employee, on='employee_id', how='left')

# Рассчитываем процент успешности по каждому оператору и продукту
success_rate = df[['employee_id', 'product_cd', 'flag']].groupby(['employee_id', 'product_cd'])['flag'].mean().reset_index()
success_rate.rename(columns={'flag': 'success_rate'}, inplace=True)

# Для каждого оператора выбираем продукт с наибольшим процентом успешности
best_product = success_rate.loc[success_rate.groupby('employee_id')['success_rate'].idxmax()]

# Добавляем столбец best_product в исходный DataFrame
emp = emp.merge(best_product[['employee_id', 'product_cd']], on='employee_id', how='left')
emp.rename(columns={'product_cd': 'best_product'}, inplace=True)

emp = emp.rename({'flag' : 'emp_mean_flag'}, axis=1)

df_plus_emp = df.merge(emp[['employee_id', 'emp_mean_flag', 'time_uspeh', 'min_age', 'max_age', 'lovely_gender_cd', 'best_product']], on='employee_id', how='left')
df_plus_emp['emps_product'] = (df_plus_emp['product_cd'] == df_plus_emp['best_product']).astype(int)

df_plus_emp['gender_match'] = (df_plus_emp['gender_cd'] == df_plus_emp['emp_gender_cd']).astype(int)
df_plus_emp['age_matching'] = ((df_plus_emp['age'] - df_plus_emp['emp_age']).abs() <= 5).astype(int)

df_plus_emp['lovely_gender_cd_encoded'] = (df_plus_emp['lovely_gender_cd'] == 'M').astype(int)

df_plus_emp['emps_lovely_age'] = ((df_plus_emp['min_age'] < df_plus_emp['age']) & (df_plus_emp['age'] < df_plus_emp['max_age'])).astype(int)

df_plus_emp['child_by_emp_and_cd_flg'] = ((df_plus_emp['child_amt'] > 0) * (df_plus_emp['emp_children_cnt'] > 0)).astype(int)

def sha_budet_rok(x):
    if x <= 25:
        return '14-25'
    elif x <= 35:
        return '26-35'
    elif x <= 45:
        return '36-45'
    elif x <= 60:
        return '45-60'
    else:
        return '60+'


df_plus_emp['age_groups_cd'] = df_plus_emp['age'].apply(lambda x: sha_budet_rok(x))
df_plus_emp['age_groups_emp'] = df_plus_emp['emp_age_new'].apply(lambda x: sha_budet_rok(x))
df_plus_emp['age_group_matching'] = (df_plus_emp['age_groups_cd'] == df_plus_emp['age_groups_emp']).astype(int)


In [173]:
import pandasql as ps

upsell = df_plus_emp[df_plus_emp['functional_cd'].isin(['Upsell'])]
upsell = upsell.reset_index()

q1 = """
SELECT * FROM upsell 
WHERE "index" NOT IN (
    SELECT "index" 
    FROM upsell 
    WHERE wo_hit_status_reason_desc IN ( 
        'Другой банк', 'Третье лицо', 'Кредит уже не нужен', 'Другое'
    )
)
"""
upsell = ps.sqldf(q1, locals())

In [131]:
import pandasql as ps

upsell2 = df_plus_emp[df_plus_emp['functional_cd'].isin(['Upsell'])]
upsell2 = upsell2.reset_index()

q2 = """
SELECT * FROM upsell2
WHERE "index" NOT IN (
    SELECT "index" 
    FROM upsell2
    WHERE wo_hit_status_reason_desc IN ( 
        'Другой банк', 'Третье лицо'
    )
)
"""
upsell2 = ps.sqldf(q2, locals())

In [132]:
import pandasql as ps

upsell4 = df_plus_emp[df_plus_emp['functional_cd'].isin(['Upsell'])]
upsell4 = upsell4.reset_index()

q4 = """
SELECT * FROM upsell4
WHERE "index" NOT IN (
    SELECT "index" 
    FROM upsell4
    WHERE wo_hit_status_reason_desc IN ( 
        'Другой банк', 'Третье лицо', 'Кредит уже не нужен'
    )
)
"""
upsell4 = ps.sqldf(q4, locals())

In [133]:
import pandasql as ps

upsell3 = df_plus_emp[df_plus_emp['functional_cd'].isin(['Upsell'])]
upsell3 = upsell3.reset_index()

# q3 = """
# SELECT * FROM upsell3
# WHERE "index" NOT IN (
#     SELECT "index" 
#     FROM upsell3
#     WHERE wo_hit_status_reason_desc IN ( 
#         'Другой банк', 'Третье лицо'
#     )
# )
# """
# upsell3 = ps.sqldf(q3, locals())

In [247]:
from scipy.stats import chi2_contingency

def KOSTIL(df, i, j):
    while i <= j:
        df = df[df['num_segment'] != i]
        i += 1
    return df

p_val = 112
ibest = -1
jbest = -1

for i in range(3, 10):
    for j in range(i+1, 10):
        df_credit = upsell.copy()
        df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, j, -1)])).astype(int)
        df_credit = KOSTIL(df_credit, i, j)
        
        # Таблица сопряженности: low_income_flg и flag (успех)
        contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])
        
        # Тест хи-квадрат
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        if p < 0.05:
            print(i, j)
        
        if p < p_val:
            p_val = p
            ibest = i
            jbest = j

print()

df_credit = upsell.copy()
df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, jbest, -1)])).astype(int)
df_credit = KOSTIL(df_credit, ibest, jbest)

# Таблица сопряженности: low_income_flg и flag (успех)
contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])

# Тест хи-квадрат
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Хи-квадрат статистика:", chi2)
print("p-value:", p)
print("i, j:", ibest, jbest)

# Интерпретация
if p < 0.05:
    print("Гипотеза подтверждена: низкий доход влияет на успешность апселла")
else:
    print("Гипотеза не подтверждена: связь между доходом и успешностью отсутствует")

3 6
3 7
3 8
3 9
4 7
4 9
5 7
5 9
6 7
6 8
6 9
7 8
7 9
8 9

Хи-квадрат статистика: 12.281119037302085
p-value: 0.000457563925466835
i, j: 3 9
Гипотеза подтверждена: низкий доход влияет на успешность апселла


In [307]:
from scipy.stats import chi2_contingency

def KOSTIL(df, i, j):
    while i <= j:
        df = df[df['num_segment'] != i]
        i += 1
    return df

df_credit = upsell[upsell['region_size'] == 'town'][upsell['num_segment'] > 0].copy()
ibest = 6
jbest = ibest

df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, jbest+1, -1)])).astype(int)
df_credit = KOSTIL(df_credit, ibest, jbest)

# Таблица сопряженности: low_income_flg и flag (успех)
contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])

# Тест хи-квадрат
chi2, p, dof, expected = chi2_contingency(contingency_table)
  
print("Хи-квадрат статистика:", chi2)
print("p-value:", p)
print("i, j:", ibest, jbest)

# Интерпретация
if p < 0.05:
    print("Гипотеза подтверждена: низкий доход влияет на успешность апселла")
else:
    print("Гипотеза не подтверждена: связь между доходом и успешностью отсутствует")

Хи-квадрат статистика: 3.309197035757145
p-value: 0.06889314699264013
i, j: 6 6
Гипотеза не подтверждена: связь между доходом и успешностью отсутствует


In [233]:
ibest = 3
jbest = 7

df_credit = upsell2[upsell2['gender_cd'] == 'M'].copy()
df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, jbest, -1)])).astype(int)
df_credit = KOSTIL(df_credit, ibest, jbest)

# Таблица сопряженности: low_income_flg и flag (успех)
contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])

# Тест хи-квадрат
chi2, p, dof, expected = chi2_contingency(contingency_table)
  
print("Хи-квадрат статистика:", chi2)
print("p-value:", p)
print("i, j:", ibest, jbest)

# Интерпретация
if p < 0.05:
    print("Гипотеза подтверждена: низкий доход влияет на успешность апселла")
else:
    print("Гипотеза не подтверждена: связь между доходом и успешностью отсутствует")

Хи-квадрат статистика: 3.15
p-value: 0.07592696298255787
i, j: 3 7
Гипотеза не подтверждена: связь между доходом и успешностью отсутствует


In [229]:
from scipy.stats import chi2_contingency

def KOSTIL(df, i, j):
    while i <= j:
        df = df[df['num_segment'] != i]
        i += 1
    return df

p_val = 112
ibest = -1
jbest = -1

for i in range(3, 10):
    for j in range(i+1, 10):
        df_credit = upsell2[upsell2['gender_cd'] == 'F'].copy()
        df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, j, -1)])).astype(int)
        df_credit = KOSTIL(df_credit, i, j)
        
        # Таблица сопряженности: low_income_flg и flag (успех)
        contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])
        
        # Тест хи-квадрат
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        if p < 0.05:
            print(i, j)
        
        if p < p_val:
            p_val = p
            ibest = i
            jbest = j

print()
# 
# df_credit = upsell2[upsell2['auto_flg'] == 0].copy()
# df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, jbest, -1)])).astype(int)
# df_credit = KOSTIL(df_credit, ibest, jbest)
# 
# # Таблица сопряженности: low_income_flg и flag (успех)
# contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])
# 
# # Тест хи-квадрат
# chi2, p, dof, expected = chi2_contingency(contingency_table)
# 
# print("Хи-квадрат статистика:", chi2)
# print("p-value:", p)
# print("i, j:", ibest, jbest)
# 
# # Интерпретация
# if p < 0.05:
#     print("Гипотеза подтверждена: низкий доход влияет на успешность апселла")
# else:
#     print("Гипотеза не подтверждена: связь между доходом и успешностью отсутствует")

3 9
4 9
5 9
6 9
7 9
8 9



In [283]:
import scipy.stats as stats

# Создаем таблицу сопряженности
# Например, таблица успехов и неудач по группам
table = pd.crosstab(upsell2['num_segment'], upsell2['flag'])
# Выполняем точный тест Фишера
oddsratio, p_value = stats.fisher_exact(table)

print("Odds Ratio:", oddsratio)
print("P-value:", p_value)

ValueError: The input `table` must be of shape (2, 2).

In [301]:
from scipy.stats import chi2_contingency

def KOSTIL(df, i, j):
    while i <= j:
        df = df[df['num_segment'] != i]
        i += 1
    return df

df_credit = upsell[upsell['region_size'] == 'msk'][upsell['num_segment'] > 0].copy()
ibest = 3
jbest = ibest

df_credit['low_income_flg'] = (df_credit['num_segment'].isin([i for i in range(12, jbest+1, -1)])).astype(int)
df_credit = KOSTIL(df_credit, ibest, jbest)

# Таблица сопряженности: low_income_flg и flag (успех)
contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])

# Тест хи-квадрат
oddsratio, p_value = stats.fisher_exact(contingency_table)

print("Odds Ratio:", oddsratio)
print("P-value:", p_value)

# Интерпретация
if p_value < 0.05:
    print("Гипотеза подтверждена: низкий доход влияет на успешность апселла")
else:
    print("Гипотеза не подтверждена: связь между доходом и успешностью отсутствует")

Odds Ratio: 1.2682926829268293
P-value: 1.0
Гипотеза не подтверждена: связь между доходом и успешностью отсутствует


In [308]:
upsell2.columns

Index(['index', 'Unnamed: 0', 'customer_id', 'age', 'gender_cd', 'region_size',
       'foreign_flg', 'entrepreneur_flg', 'auto_flg', 'traveler_flg',
       'partnership_flg', 'child_amt', 'realty_type', 'segment_cd',
       'bundle_nm', 'confirmed_income', 'predicted_income', 'product_cd',
       'functional_cd', 'finish_dttm', 'state_talk_time_sec',
       'wo_hit_status_result_desc', 'wo_hit_status_reason_desc', 'employee_id',
       'emp_age', 'emp_age_new', 'emp_gender_cd', 'emp_children_cnt',
       'emp_work_months_cnt', 'emp_citizenship_country_cd',
       'emp_marital_status_cd', 'flag', 'day_of_week', 'num_of_week',
       'child_flg', 'income', 'segment_number', 'num_segment', 'month',
       'hours', 'season'],
      dtype='object')

In [314]:
upsell2.pivot_table(index='customer_id', values='flag', aggfunc='count').sort_values(by='flag')

,flag
customer_id,
591235124,1
528365506,1
528648661,1
529208008,1
529391325,1
...,...
582731576,1
526474864,2
383883359,2


In [320]:
upsell2['product_cd'].value_counts()

product_cd
Cash Loan       196
Secured Loan    159
Credit Cards    141
Name: count, dtype: int64

In [223]:
people = df_plus_emp[df_plus_emp['region_size'].isin(['town'])][['customer_id', 'num_segment']].groupby(by='customer_id').max()

percentile75 = np.percentile(people['num_segment'].dropna(), 61.5)

In [224]:
percentile75

np.float64(7.0)

In [294]:
from scipy.stats import chi2_contingency
import scipy.stats as stats
from statsmodels.stats.contingency_tables import Table2x2
def pomogite(df_credit):
    percentile75 = np.percentile(df_credit['num_segment'].dropna(), 61.5)
    df_credit['low_income_flg'] = (df_credit['num_segment'] >= percentile75).astype(int)
    
    # Таблица сопряженности: low_income_flg и flag (успех)
    contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])
    
    # Тест хи-квадрат
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return p

# percentile75 = np.percentile(df_credit['num_segment'].dropna(), 61.5)
def fisher(df_credit):
    df_credit['low_income_flg'] = (df_credit['num_segment'] >= percentile75).astype(int)
    
    # Таблица сопряженности: low_income_flg и flag (успех)
    contingency_table = pd.crosstab(df_credit['low_income_flg'], df_credit['flag'])
    table = Table2x2(contingency_table.values)
    # Тест хи-квадрат
    oddsratio, p = stats.fisher_exact(contingency_table)
    ci_low, ci_high = table.oddsratio_confint(alpha=0.05)
    print(f"Доверительный интервал: \n 95%CI: ({ci_low, ci_high})\n\nОтношение шансов: {oddsratio}\n\np_value: {p}")
    return oddsratio, p


In [295]:
pomogite(upsell[upsell['region_size'].isin(['town', 'rural'])])

np.float64(0.0589388351552022)

In [296]:
fisher(upsell[upsell['region_size'].isin(['town', 'rural'])])

Доверительный интервал: 
 95%CI: ((np.float64(1.1717290366424065), np.float64(4.304389634420074)))

Отношение шансов: 2.2457912457912457

p_value: 0.015945763734293957


(np.float64(2.2457912457912457), np.float64(0.015945763734293957))

In [238]:
def test(XAAXAXAXA):
    print(float(fisher(XAAXAXAXA[XAAXAXAXA['region_size'].isin(['town', 'rural'])])), '\n', float(pomogite(XAAXAXAXA[XAAXAXAXA['region_size'].isin(['town', 'rural'])])))

In [239]:
float(fisher(upsell[upsell['region_size'].isin(['town', 'rural'])]))

0.015945763734293957

In [274]:
upsell4['nedvizhemost'] = ((upsell4['auto_flg'] == 1) | (upsell4['realty_type'] == 'Квартира')).astype(int)

In [278]:
test(upsell4)

0.0437695516430974 
 0.1032982474867485


In [267]:
upsell4[upsell4['gender_cd'] == 'F'][upsell4['region_size'].isin(['town', 'rural'])]

,index,Unnamed: 0,customer_id,age,gender_cd,region_size,foreign_flg,entrepreneur_flg,auto_flg,traveler_flg,...,best_product,emps_product,gender_match,age_matching,lovely_gender_cd_encoded,emps_lovely_age,child_by_emp_and_cd_flg,age_groups_cd,age_groups_emp,age_group_matching
2,9,213,424440108,19.0,F,rural,0.0,0.0,1.0,0.0,...,Debit Cards,0,0,0,1,0,0,14-25,26-35,0
7,23,875,123838898,30.0,F,rural,0.0,0.0,1.0,0.0,...,Cash Loan,0,0,0,1,0,0,26-35,36-45,0
9,27,1085,451353576,31.0,F,town,0.0,0.0,0.0,0.0,...,Secured Loan,1,0,1,0,0,0,26-35,26-35,1
11,41,1436,296891168,34.0,F,town,0.0,0.0,0.0,0.0,...,Cash Loan,0,0,0,1,0,0,26-35,14-25,0
13,43,1498,335074068,34.0,F,rural,0.0,0.0,1.0,0.0,...,Debit Cards,0,0,1,1,1,0,26-35,26-35,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,14060,17691,284871735,26.0,F,town,0.0,0.0,1.0,0.0,...,Secured Loan,0,0,1,1,0,0,26-35,26-35,1
929,14286,18087,135694051,37.0,F,town,0.0,0.0,0.0,0.0,...,Cash Loan,1,0,1,0,0,0,36-45,36-45,1
930,14290,18094,469195389,18.0,F,town,0.0,0.0,0.0,0.0,...,Cash Loan,1,0,0,1,0,0,14-25,14-25,1
951,14590,18600,318849366,37.0,F,town,0.0,0.0,1.0,0.0,...,Cash Loan,1,0,0,1,0,0,36-45,45-60,0


In [298]:
upsell4.pivot_table(index='region_size', values='flag', aggfunc='count')

,flag
region_size,
msk,66
rural,101
town,427
urban,155


In [300]:
df_plus_emp.pivot_table(index='region_size', values='flag', aggfunc='count')

,flag
region_size,
msk,1245
rural,964
town,5087
urban,2175


In [301]:
upsell4.pivot_table(index='region_size', values='flag', aggfunc='count')

,flag
region_size,
msk,66
rural,101
town,427
urban,155
